In [388]:
import pandas as pd
import math


# User Inputs

pokename = 'Blastoise'
attack_stat = 2
defense_stat = 13
stamina_stat = 15


### Creating a table which contains all the possible IV permutations

In [389]:
page = []

for i in range(0,16,1):
    for a in range(0,16,1):    
        for b in range(0,16,1):
            page.append([i,a,b])

iv_Permu = pd.DataFrame(page)

iv_Permu.columns = ['attack_iv', 'defense_iv', 'stamina_iv']

#print(iv_Permu.shape) # 16x16x16 = 4096 as a check that we covered all possible permutations
#print(iv_Permu.head(10))
#print(iv_Permu.tail(10))

#### Importing CP to Level table 

In [390]:
multiplier = pd.read_excel('excelfiles\\multiplier.xlsx')
multiplier.columns = ['cp_multiplier','level']
#print(multiplier.head(10))

#### Importing all pokemon base stats

In [391]:
pokestats = pd.read_excel('excelfiles\\pokestats.xlsx')
pokestats.columns  = ['name','attack','defense', 'stamina']
pokestats = pokestats.set_index('name')


#print(pokestats.head(10))

In [392]:
current_poke = pokestats.loc[pokename]

In [393]:

iv_Permu['base_attack'] = current_poke[0]
iv_Permu['base_defense'] = current_poke[1]
iv_Permu['base_stamina']  = current_poke[2]
    

iv_Permu['total_attack'] = iv_Permu['base_attack'] + iv_Permu['attack_iv']
iv_Permu['total_defense'] = iv_Permu['base_defense'] + iv_Permu['defense_iv']
iv_Permu['total_stamina'] = iv_Permu['base_stamina'] + iv_Permu['stamina_iv']
    
#print(iv_Permu.head(10))


In [394]:
multiplier = multiplier.set_index('level')


In [395]:
multiplier = multiplier[10:].sort_index(ascending=False)

In [396]:
#print(multiplier.iloc[1]) #returns row 1 as a series object
#print(multiplier.iloc[1][0]) #access the series object to obtain the value
iv_Permu['max_level'] = 0
iv_Permu['max_cp']= 0
iv_Permu['cpm'] = 0



for i in range(len(iv_Permu)):
    counter = 40
    attack_factor = iv_Permu['total_attack'][i]
    defense_factor = iv_Permu['total_defense'][i] ** .5
    stamina_factor = iv_Permu['total_stamina'][i] ** .5
    cp_factor = multiplier['cp_multiplier'][counter] ** 2

    cp_calc = (attack_factor * defense_factor * stamina_factor * cp_factor) / 10
    

    while cp_calc > 1501:
        counter -= .5
        cp_factor = multiplier['cp_multiplier'][counter] ** 2
        cp_calc = (attack_factor * defense_factor * stamina_factor * cp_factor) / 10
        
    else:
        iv_Permu['max_level'][i] = counter
        iv_Permu['max_cp'][i]= cp_calc // 1
        iv_Permu['cpm'][i] = (cp_factor ** .5) 
        
        


C:\Users\abeng\AppData\Local\Temp\ipykernel_24616\3361785002.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iv_Permu['max_cp'][i]= cp_calc // 1
C:\Users\abeng\AppData\Local\Temp\ipykernel_24616\3361785002.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iv_Permu['cpm'][i] = (cp_factor ** .5)


In [397]:
iv_Permu['final_attack'] = round(iv_Permu['total_attack'] * iv_Permu['cpm'],2)
iv_Permu['final_defense'] = round(iv_Permu['total_defense'] * iv_Permu['cpm'],2)
iv_Permu['final_stamina'] = (iv_Permu['total_stamina'] * iv_Permu['cpm']) // 1



In [398]:
iv_Permu['stat_prod'] = round((iv_Permu['final_attack'] * iv_Permu['final_defense'] * iv_Permu['final_stamina']) / 1000,1)

In [399]:
final = iv_Permu.sort_values('stat_prod',ascending=False).head(10).reset_index(drop=True)

final['rank'] = 0

for i in range(len(final)):
    final['rank'][i] = i + 1


C:\Users\abeng\AppData\Local\Temp\ipykernel_24616\1382504007.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['rank'][i] = i + 1


In [400]:
final.head(15)

,attack_iv,defense_iv,stamina_iv,base_attack,base_defense,base_stamina,total_attack,total_defense,total_stamina,max_level,max_cp,cpm,final_attack,final_defense,final_stamina,stat_prod,rank
0,1,15,15,171,207,188,172,222,203,23.0,1498,0.640653,110.19,142.22,130.0,2037.3,1
1,2,13,15,171,207,188,173,220,203,23.0,1500,0.640653,110.83,140.94,130.0,2030.6,2
2,0,14,10,171,207,188,171,221,198,23.5,1500,0.647581,110.74,143.12,128.0,2028.7,3
3,1,14,15,171,207,188,172,221,203,23.0,1495,0.640653,110.19,141.58,130.0,2028.1,4
4,0,12,12,171,207,188,171,219,200,23.5,1500,0.647581,110.74,141.82,129.0,2026.0,5
5,0,15,15,171,207,188,171,222,203,23.0,1489,0.640653,109.55,142.22,130.0,2025.4,6
6,2,14,14,171,207,188,173,221,202,23.0,1500,0.640653,110.83,141.58,129.0,2024.2,7
7,0,10,13,171,207,188,171,217,201,23.5,1497,0.647581,110.74,140.53,130.0,2023.1,8
8,0,15,9,171,207,188,171,222,197,23.5,1499,0.647581,110.74,143.76,127.0,2021.8,9
9,1,15,14,171,207,188,172,222,202,23.0,1494,0.640653,110.19,142.22,129.0,2021.6,10


In [401]:
clean_final = final[['rank', 'attack_iv','defense_iv','stamina_iv','max_cp','max_level','final_attack', 'final_defense','final_stamina', 'stat_prod']]
clean_final

,rank,attack_iv,defense_iv,stamina_iv,max_cp,max_level,final_attack,final_defense,final_stamina,stat_prod
0,1,1,15,15,1498,23.0,110.19,142.22,130.0,2037.3
1,2,2,13,15,1500,23.0,110.83,140.94,130.0,2030.6
2,3,0,14,10,1500,23.5,110.74,143.12,128.0,2028.7
3,4,1,14,15,1495,23.0,110.19,141.58,130.0,2028.1
4,5,0,12,12,1500,23.5,110.74,141.82,129.0,2026.0
5,6,0,15,15,1489,23.0,109.55,142.22,130.0,2025.4
6,7,2,14,14,1500,23.0,110.83,141.58,129.0,2024.2
7,8,0,10,13,1497,23.5,110.74,140.53,130.0,2023.1
8,9,0,15,9,1499,23.5,110.74,143.76,127.0,2021.8
9,10,1,15,14,1494,23.0,110.19,142.22,129.0,2021.6


In [402]:
results = clean_final.loc[(final['attack_iv'] == attack_stat) & (final['defense_iv'] == defense_stat) & (final['stamina_iv'] == stamina_stat)]

In [403]:
clean_final.head(15)


,rank,attack_iv,defense_iv,stamina_iv,max_cp,max_level,final_attack,final_defense,final_stamina,stat_prod
0,1,1,15,15,1498,23.0,110.19,142.22,130.0,2037.3
1,2,2,13,15,1500,23.0,110.83,140.94,130.0,2030.6
2,3,0,14,10,1500,23.5,110.74,143.12,128.0,2028.7
3,4,1,14,15,1495,23.0,110.19,141.58,130.0,2028.1
4,5,0,12,12,1500,23.5,110.74,141.82,129.0,2026.0
5,6,0,15,15,1489,23.0,109.55,142.22,130.0,2025.4
6,7,2,14,14,1500,23.0,110.83,141.58,129.0,2024.2
7,8,0,10,13,1497,23.5,110.74,140.53,130.0,2023.1
8,9,0,15,9,1499,23.5,110.74,143.76,127.0,2021.8
9,10,1,15,14,1494,23.0,110.19,142.22,129.0,2021.6


In [404]:
results

,rank,attack_iv,defense_iv,stamina_iv,max_cp,max_level,final_attack,final_defense,final_stamina,stat_prod
1,2,2,13,15,1500,23.0,110.83,140.94,130.0,2030.6
